## АвтОбрЕЯ: домашнее задание 1
Елена Зайцева (БКЛ201)

In [34]:
import requests
from fake_useragent import UserAgent
import random
import sqlite3
from bs4 import BeautifulSoup

In [35]:
import nltk
from nltk.tokenize import word_tokenize

In [36]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [37]:
from collections import Counter

In [38]:
from nltk.corpus import stopwords
sw = stopwords.words('russian')

In [39]:
from sklearn.metrics import accuracy_score

In [40]:
# парсинг отзывов с кинопоиска -- одностраничный
def parse_reviews(url):
    session = requests.session()
    ua = UserAgent(verify_ssl=False)
    req = session.get(url, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    reviews = soup.find_all('div', {'class': 'brand_words'})
    reviews_prs = ''
    for i in range(0, len(reviews)):
        reviews_prs += ' ' + reviews[i].find('p').text
    return reviews_prs

In [41]:
# очистка от знаков препинания и стоп-слов
def clean_tokenizer(reviews_prs):
    words_cltok = []
    for i in word_tokenize(reviews_prs.lower()):
        if i.strip('…').isalpha() and i.strip('…') not in sw:
            words_cltok.append(morph.parse(i)[0].normal_form)
    return words_cltok

In [42]:
# фильтр слов по частотности
def word_filter(words_cltok):
    prs_dct = dict(Counter(words_cltok).most_common())
    prs_filtered = []
    for key, value in prs_dct.items():
        if value > 2:
            prs_filtered.append(key)
    return prs_filtered

In [43]:
# парсинг с нескольких страниц
# polarity -- good или bad
# num -- номер страницы
def get_pages(num, polarity):
    pol_reviews = ''
    for i in range(1, num+1):
        url = f'https://www.kinopoisk.ru/reviews/type/comment/status/{polarity}/period/month/page/{num}/#list'
        pol_reviews += ' ' + parse_reviews(url)
    return pol_reviews

In [44]:
def get_words(all_reviews):
    all_words = clean_tokenizer(all_reviews)
    all_filtered = word_filter(all_words)
    return all_filtered

In [45]:
# работает с двумя отфильтрованными множествами слов
def unique_words(base, unneeded):
    set_1 = set(base)
    set_2 =set(unneeded)
    needed = set_1 - set_2
    return needed

In [46]:
# парсинг отзывов по отдельности
# функция для отзывов, на которых будет определяться accuracy
def parse_separated(url):
    sess = requests.session()
    req = sess.get(url, headers={'User-Agent': UserAgent(verify_ssl=False).random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    revs = soup.find_all('div', {'class': 'brand_words'})
    revs_sep = []
    for i in range(0, len(revs)):
        revs_sep.append(revs[i].find('p').text)
    return revs_sep

In [47]:
# функция определяет отрицательный ли отзыв, положительный или нейтральный
def review_polarity(one_review, pos_set, neg_set):
    words_one = clean_tokenizer(one_review)
    pos = 0
    neg = 0
    for w in words_one:
        if w in pos_set:
            pos += 1
        elif w in neg_set:
            neg += 1
    if pos > neg:
        return 'pos'
    elif pos < neg:
        return 'neg'
    else:
        return 'neutr'

### 1. и 2. Скачивание даты, токенизация, приведение к начальной форме, фильтрация по частотности

In [50]:
# сбор слов для "тонального словаря" -- отдельно из положительных и из отрицательных отзывов
positive = get_words(get_pages(27, 'good'))
negative = get_words(get_pages(27, 'bad'))

10 отзывов на одной странице -- всего 540 (270 положительных и 270 отрицательных) отзывов.

### 3. Составление двух множеств

In [52]:
# слова, которые встречаются только в положительных или только в отрицательных отзывах
positive_uniq = unique_words(positive, negative)
negative_uniq = unique_words(negative, positive)

### 4. Скачивание отзывов для проверки, подсчет accuracy

7 отрицательных отзывов и 10 положительных.

In [53]:
pos_test = parse_separated('https://www.kinopoisk.ru/reviews/type/comment/status/good/period/month/page/28/#list')
neg_test = parse_separated('https://www.kinopoisk.ru/reviews/type/comment/status/bad/period/month/page/28/#list')
all_test = pos_test + neg_test

In [54]:
rev_true = ['pos'] * len(pos_test) + ['neg'] * len(neg_test)
rev_pred = []
for r in all_test:
    rev_pred.append(review_polarity(r, positive_uniq, negative_uniq))

In [55]:
accuracy_score(rev_true, rev_pred)

0.47058823529411764

### 5. Что можно добавить

- можно добавить исправление орфографических ошибок, опечаток (чтобы 'хроший' приравнивался к 'хороший' или 'офигенный' к 'афигенный' -- хотя в последней паре скорее не исправление ошибки, а отнесение к какому-то одному общему варианту)
- учитывать смайлики (если встречаются), определять к какому типу отзывов они относятся
- сохранять отрицания (чтобы различать 'понравился' и 'не понравился' и не различать 'не плохой' и 'неплохой')